In [2]:
from overlaps import (
    read_overlaps_from_file, merge_multiple_predictions, compute_crispr_overlaps
)
from misclassifications import get_misclassifications, label_misclassifications, LABELERS, groupby_gene
from main import OVERLAP_FILENAME
from df_loader import CrisprDFLoader, PredDFLoader
from schema import DFSchema
import pandas as pd


In [3]:
CRISPR_FILENAME = "/oak/stanford/groups/engreitz/Projects/Benchmarking/CRISPR_data/EPCrisprBenchmark_ensemble_data_GRCh38.tsv.gz"
PRED_FILENAME = "/oak/stanford/groups/engreitz/Users/atan5133/abc_run_comparisons/results_10_27_dev/Predictions/EnhancerPredictionsAllPutative.tsv.gz"
OVERLAP_FILENAME = "crispr_pred_overlaps_noqnorm.csv"
ABC_THRESHOLD = 0.024
TSS_REF_FILE = "resources/genome_annotations/RefSeqCurated.170308.bed.CollapsedGeneBounds.hg38.TSS500bp.bed"

In [4]:
# overlaps = read_overlaps_from_file(OVERLAP_FILENAME)
pred_df = PredDFLoader(PRED_FILENAME, TSS_REF_FILE, ABC_THRESHOLD).load()
crispr_df = CrisprDFLoader(CRISPR_FILENAME, TSS_REF_FILE).load()
overlap_df = compute_crispr_overlaps(
    crispr_df, pred_df
)
overlaps = merge_multiple_predictions(overlap_df, ABC_THRESHOLD)

/oak/stanford/groups/engreitz/Users/atan5133/.conda/envs/error_analysis/lib/python3.9/site-packages/bioframe/core/specs.py:150: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  pd.api.types.is_categorical_dtype(chrom_dtype),
/oak/stanford/groups/engreitz/Users/atan5133/.conda/envs/error_analysis/lib/python3.9/site-packages/bioframe/core/specs.py:150: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  pd.api.types.is_categorical_dtype(chrom_dtype),
/oak/stanford/groups/engreitz/Users/atan5133/.conda/envs/error_analysis/lib/python3.9/site-packages/bioframe/ops.py:514: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_input_2[overlap_df_idxs[:, 1] =

NameError: name 'overlaps' is not defined

In [ ]:
misclass_df = get_misclassifications(overlaps)
label_misclassifications(misclass_df)
pd.set_option('display.max_rows', 100) 

In [ ]:
for labeler in LABELERS:
    print(labeler.summarize_category_count(misclass_df))
    print()

In [ ]:
false_pos = len(misclass_df[(misclass_df["EnhancerSize_Category"] == "Large") & (misclass_df["FalsePos_Category"] == True)])
false_neg = len(misclass_df[(misclass_df["EnhancerSize_Category"] == "Large") & (misclass_df["FalseNeg_Category"] == True)])
print(f"False pos: {false_pos}\nFalse neg: {false_neg}")

In [ ]:
# Look at MYC classification
def print_target_gene_misclassification(target_gene):
    incorrect = len(misclass_df[misclass_df[DFSchema.TARGET_GENE + DFSchema.CRISPR_SUFFIX] == target_gene])
    total_pairs = len(overlaps[overlaps[DFSchema.TARGET_GENE + DFSchema.CRISPR_SUFFIX] == target_gene])
    print(f"Misclassified {incorrect}/{total_pairs} eg pairs for {target_gene}.\n{int(100 * incorrect/total_pairs)}% incorrect")

print_target_gene_misclassification("MYC")
print_target_gene_misclassification("PLP2")

In [ ]:
# Example queries
print(misclass_df["FalsePos_Category"].value_counts().filter([True]))
print()
print(len(misclass_df.query("FalseNeg_Category == True")))

In [ ]:
sizes = pred_df["end"] - pred_df["start"]
num_normal = (sizes == 500).sum()
num_large = (sizes > 500).sum()

In [ ]:
num_normal

In [ ]:
num_large